In [1]:
import requests
import os
import numpy as np
from netCDF4 import Dataset
import datetime as dt
import shutil
import array
import pandas as pd

In [2]:
def change_val_1D_char(ds,varname,new_val):
    var=ds.variables[varname]
    old_val=var[:].tobytes().decode().strip()
    
    print("Modifying ",varname," from ", old_val," to ",new_val)

    new_array=np.chararray(var.shape)
    new_array[:]=" " 
    for i in range(len(new_val)):
        new_array[i]=new_val[i]

    var[:]=new_array

In [3]:
def change_val_2D_char(ds,varname,dim_level,new_val):
    var=ds.variables[varname]
    old_val=var[dim_level,:].tobytes().decode().strip()
    print("Modifying ",varname," for dim_level ", dim_level, "from ", old_val," to ",new_val)

    new_type_array=np.chararray(var[dim_level,:].shape)
    new_type_array[:]=" " 
    for i in range(len(new_val)):
        new_type_array[i]=new_val[i]
    var[dim_level,:]=new_type_array[:]

In [4]:
def change_TPN_name(TPN,old_name,new_name):
    i_TPN  = [index for (index, item) in enumerate(list(TPN)) if item == old_name]
    TPN[i_TPN]=new_name
    return TPN

In [5]:
def delete_TP(CYCLE_NUMBER,TPN,TPV,param_name_to_delete):
    i_TPN_to_delete  = [index for (index, item) in enumerate(list(TPN)) if item == param_name_to_delete]
    TPN=np.delete(TPN,i_TPN_to_delete)
    TPV=np.delete(TPV,i_TPN_to_delete)
    CYCLE_NUMBER=np.delete(CYCLE_NUMBER,i_TPN_to_delete)
    return CYCLE_NUMBER,TPN,TPV

In [6]:
def delete_cycles(CYCLE_NUMBER,TPN,TPV,bad_cycles_to_delete):
    i_cycle_to_delete  = [index for (index, item) in enumerate(list(CYCLE_NUMBER)) if item in bad_cycles_to_delete]
    TPN=np.delete(TPN,i_cycle_to_delete)
    TPV=np.delete(TPV,i_cycle_to_delete)
    CYCLE_NUMBER=np.delete(CYCLE_NUMBER,i_cycle_to_delete)

    return CYCLE_NUMBER,TPN,TPV

In [7]:
def change_TPN_date_unit(TPN,TPV,old_name,new_name,old_format,new_format):
    i_TPN  = [index for (index, item) in enumerate(list(TPN)) if item == old_name]
    for i in i_TPN:
        val=list(TPV[i].strip())
       
        # Note here that it is very specific 
        if (old_format=="DD/MM/YYYY HH:MM:SS") & (new_format == "YYYYMMDDHHMMSS") :
            new_val=[None] * 14
            new_val[0:4] = val[6:10] 
            new_val[4:6] = val[3:5]
            new_val[6:8] = val[0:2]
            new_val[8:10] = val[11:13]
            new_val[10:12] = val[14:16]
            new_val[12:14] = val[17:19]
            #print("val,new_val,str(new_val)")
            #print(val,new_val,''.join(new_val))
            TPV[i]=''.join(new_val)
            TPN[i]=new_name
        elif (old_format=="HH:MM") & (new_format == "HHMM"):
            new_val=[None] * 4
            if len(TPV[i].strip())<=5: # some basic check on input format
                new_val[0:2] = val[0:2] 
                new_val[2:4] = val[3:5]
                TPV[i]=''.join(new_val)
                TPN[i]=new_name
        elif (old_format=="YYYY/MM/DD HH:MM") & (new_format == "YYYYMMDDHHMMSS"):
            new_val=[None] * 14
            if len(TPV[i].strip())>=16: # some basic check on input format
                new_val[0:4] = val[0:4] 
                new_val[4:6] = val[5:7]
                new_val[6:8] = val[8:10]
                new_val[8:10] = val[11:13]
                new_val[10:12] = val[14:16]
                new_val[12:14] = "00"
                TPV[i]=''.join(new_val)
                TPN[i]=new_name
        else:
            print("your format change is not yet coded: TO DO")
        
    return TPN,TPV


In [8]:
def rebuild_complete_from_dot_profile(lcycle,lparam,lvalue,lunit):

    n_lines=np.size(lcycle)

    ii=0
    nb_skipped=0
    for iline in range(n_lines):
        icycle=lcycle[iline]
        #print(icycle,~icycle.startswith("999"))
        iparam=lparam[iline]
        itechparam=tech_argo_name[iparam]
        if (icycle.startswith("999") | (itechparam == "NORECORD")):
            nb_skipped=nb_skipped+1
            #print("line skipped:",icycle,iparam,itechparam)
        else:
            
            ivalue=lvalue[iline]
            iunit=lunit[iline]

            ## A little patching work for known errors

            # First a little check if value can be cast into a float 
            # (this should not often happen: thus the try except method is used)
            value_is_a_float=True
            try:
                ivalue_flt=float(ivalue)
            except:
                value_is_a_float=False
            
            # case when there have been an error in the declared unit in the .profile file
            if value_is_a_float:
                if (iparam == "xmit_pump_battery_voltage") & (float(ivalue) > 50): 
                    ivalue=float(ivalue)/100
                # case when there have been an error in the conversion and/or unit in the .profile file
                if (iparam == "xmit_pump_battery_current") & (float(ivalue) <30)  & (iunit=="[mA]"): 
                    ivalue=float(ivalue)/10
                # case when units/values are consistent in the .profile file (convert from mamps to amps)
                if (iparam == "xmit_pump_battery_current") & (float(ivalue) >=30) & (iunit=="[mA]"): 
                    ivalue=float(ivalue)/1000
                # convert from psi to mbar
                if (iparam == "xmit_internal_vacuum") & (float(ivalue) < 100): 
                    ivalue=float(ivalue)*68.9475729318
                # convert from psi to mbar
                if (iparam == "xmit_tube_pressure_surface") & (float(ivalue) < 100): 
                    ivalue=float(ivalue)*68.9475729318
            # Deal with all cases for FLAG_IceDetected_LOGICAL and upcast_status:
            if (iparam == "xmit_upcast_status") & ((ivalue=="3") | (ivalue=="ICE")):
                ori_val=ivalue
                ivalue = 1
                print(iparam,":",ori_val,"stored as ",ivalue)
            elif (iparam == "xmit_upcast_status"):
                ori_val=ivalue
                ivalue = 0
                print(iparam,":",ori_val,"stored as ",ivalue)

            #print(iparam,ivalue)
            if ii==0:
                TPN=np.array([itechparam])
                TPV=np.array([ivalue],dtype=object)
                CYCLE_NUMBER=np.array([int(icycle)])
                ii=1
            else:
                TPN=np.append(TPN,itechparam)
                TPV=np.append(TPV,ivalue)
                CYCLE_NUMBER=np.append(CYCLE_NUMBER,int(icycle))

    
    return CYCLE_NUMBER,TPN,TPV
    

In [21]:
## CASE 2 floats
case = 2

liste_wmos=['7900069','7900070','7900073','7900080','7900082','7900083','7900086','7900093','7900097',
            '6900270','6900271','1900517','1900518','6900325','6900326','6900327','6900542','6900543',
            '6900832','6900833','6900590','6900591','6900592','6900593','6900594','6900595','6900596',
            '6900597','6900809','6900808']

liste_sn=['awi_0064','awi_0065','awi_0068','awi_0075','awi_0077','awi_0078','awi_0081','awi_0088','awi_0092',
          'nemo_0016','nemo_0017','nemo_0018','nemo_0019','nemo_0020','nemo_0021','nemo_0022','nemo_0034','nemo_0057',
          'nemo_0061','nemo_0063','nemo_0070','nemo_0071','nemo_0072','nemo_0073','nemo_0074','nemo_0075','nemo_0076',
          'nemo_0077','nemo_0078','nemo_0079']


## CASE 4 floats
case = 4
#liste_wmos=['7900465','7900466','6901387','7900346','6900888','7900357','7900359','6900881','7900361','7900362','7900363',
#'7900364','7900365','7900366','7900367','7900344','7900354','7900351','7900349','7900341','7900343','7900347','7900340',
#'7900348','7900350','7900345','7900353','7900352','7900356','7900355','6900882','7900368','7900412','7900413','7900414','7900369','7900370','7900372','7900373','7900374',
#'7900375','7900376','6900883','6901902','7900377','7900378','7900379','6900884','7900380','7900381','7900382','7900383',
#'7900384','7900385','7900400','7900401','7900402','7900403','7900404','7900406','7900407','7900408',
#'7900409','7900410','7900411']
#liste_sn=['nemo_0144','nemo_0145','nemo_0146','nemo_0149','nemo_0175','nemo_0176','nemo_0178','nemo_0180','nemo_0181','nemo_0182','nemo_0184',
#'nemo_0185','nemo_0186','nemo_0187','nemo_0188','nemo_0126','nemo_0127','nemo_0128','nemo_0129','nemo_0130','nemo_0131','nemo_0132','nemo_0133','nemo_0134',
#'nemo_0136','nemo_0137','nemo_0139','nemo_0140','nemo_0141','nemo_0142','nemo_0190','nemo_0191','nemo_0208','nemo_0209','nemo_0210','nemo_0211','nemo_0212','nemo_0214','nemo_0216',
#'nemo_0217','nemo_0218','nemo_0219','nemo_0220','nemo_0223','nemo_0227','nemo_0228','nemo_0229','nemo_0230','nemo_0231','nemo_0232','nemo_0233','nemo_0234','nemo_0235','nemo_0236','nemo_0237','nemo_0238',
#'nemo_0239','nemo_0240','nemo_0241','nemo_0243','nemo_0244','nemo_0245','nemo_0246','nemo_0247','nemo_0248']

liste_wmos=['6900878','6900880']
liste_sn=['nemo_0183','nemo_0192']
lot = "lot_013"


liste_wmos=['6900885','7900464','7900467',
'7900468','7900471','7900472','7900473','7900477','7900488','7900489','7900491','7900492','7900493']
liste_sn=['nemo_0135','nemo_0270','nemo_0273','nemo_0274','nemo_0277','nemo_0278','nemo_0279','nemo_0283','nemo_0295',
'nemo_0296','nemo_0271','nemo_0272','nemo_0173']
lot = "lot_014"

## CASE 3 floats
#case = 3
#liste_wmos=['7900122','7900123','7900126','7900155','7900156','7900157','7900158','7900159',
#'7900160','7900161','7900162','7900232','7900226','7900218','7900227','7900231','7900223',
#'7900222','7900234','7900228','7900229','7900233','7900230','6900584','6900585','6900587',
#'6900588','6900589']
#liste_sn=['awi_0093','awi_0094','awi_0097','awi_0106','awi_0107','awi_0108','awi_0109','awi_0110','awi_0111','awi_0112',
#'awi_0113','awi_0120','awi_0121','awi_0123','awi_0124','awi_0125','awi_0126','awi_0127',
#'awi_0129','awi_0130','awi_0132','awi_0133','awi_0134','nemo_0048','nemo_0059','nemo_0067','nemo_0068','nemo_0069']

# The 2 floats here below must be treated separately: there are missing cycles in the archive directory I have: there must be a patch for these missing cycles.
# 7900224 is missing cycles [16, 21, 26, 28,30]
# 7900219 is missing cycles [2, 3,4,5,6,7,8,9,10,11,12,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]
#liste_wmos=['7900224','7900219']
#liste_sn=['awi_0122','awi_0128']
#lot = "lot_010"


## CASE 5 floats
#case = 5
#liste_wmos=['6900815','6900816','6900817','6900818','6900819',
#'6900823','6900824','6900825','6900826','6900827','3901085','3901086','3901087','6900820',
#'6900821','6900822','1901350','1901351','6900601','6900810','6900813','6900814','6900828','6900829',
#'6900830','6900811','6900812','6900599','6900600']
#liste_sn=['nemo_0081','nemo_0082','nemo_0083','nemo_0084','nemo_0085','nemo_0086','nemo_0087','nemo_0088','nemo_0089',
#'nemo_0090','nemo_0091','nemo_0092','nemo_0093','nemo_0095','nemo_0096','nemo_0097','nemo_0099','nemo_0100',
#'nemo_0101','nemo_0102','nemo_0103','nemo_0104','nemo_0105','nemo_0106','nemo_0107','nemo_0109','nemo_0110',
#'nemo_0111','nemo_0112']
#lot="lot_011"


## CASE 1 floats
#case = 1
#liste_wmos=['1900381','1900382','6900729']
#liste_sn=['awi_0060','awi_0061','awi_0062']
#lot="lot_012"

## CASE no TECH VALUES floats
liste_wmos=['1900380','1901349','6900831','6900886','6900887','6901057','6901388','6902041','7900221',
'7900338','7900339','7900342','7900358','7900360','7900371','7900405']
liste_sn=liste_wmos
lot="lot_015"

In [22]:
print(case)

4


In [23]:
if case == 1:
    # initiated from case 2
    # quite a strange case as the tech.nc file contains a few computed parameter related to time. But the accuracy is questionable. 
    # There is in particular a time correction applied and a floattimeseconds that is not exactly matching the .profile information.
    # in addition, there are numbers regarding transmissions and Argos positioning that are not in the technical sections and not indicated either
    # on more recent floats...

    print("Loading matching nemo to argo technical parameter names for case 1")
    tech_argo_name={}
    tech_argo_name["xmit_ascent_starttime"]="CLOCK_StartAscentToSurface_seconds"
    tech_argo_name["xmit_descent_starttime"]="CLOCK_StartDescentToProfile_seconds"
    tech_argo_name["xmit_surfacingtime"]="NORECORD" # In the manual: "Surfacing time at 100 dbar": this is not 
    # exactly "EndAscentToSurface". I have not found a tech term for this particular one.
    tech_argo_name["xmit_end_of_profile_time"]="CLOCK_EndAscentToSurface_seconds" # In the manual: "End of profile time
    tech_argo_name["xmit_time_from_starttime"]="CLOCK_FloatTime_SECONDS" 
    tech_argo_name["xmit_time_from_startup"]="CLOCK_FloatTime_SECONDS" 
    #(after last sample or abortion due to ice)"
    tech_argo_name["xmit_ice_detect_count"]="FLAG_IceDetected_COUNT"
    tech_argo_name["xmit_internal_vacuum"]="PRESSURE_InternalVacuumAtSurface_mbar" # With unit conversion
    tech_argo_name["xmit_park_piston_position"]="POSITION_PistonPark_COUNT"
    tech_argo_name["xmit_park_pressure_median"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, closest would be deprecated PRES_ParkMean_dbar
    tech_argo_name["xmit_piston_position_at_surf"]="POSITION_PistonSurface_COUNT"
    tech_argo_name["xmit_profile_length1"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length2"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length3"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length4"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length5"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_piston_position"]="POSITION_PistonProfile_COUNT"# maybe to change as TECH param defines as "piston position at maximum profile pressure"
    tech_argo_name["xmit_pump_battery_current"]="CURRENT_MotorStartProfile_amps" # with unit conversion. N.B units in .profile seem wrong (to multiply by 100 to get mamps or to divide by 10 to get amps), but not everywhere ...
    # Difficult to assess for this one: the manual says xmit_pump_battery_current = "hydraulic current during first ascent (during initialization of motor)".
    # the name has been changed between case 2 and 4: xmit_pump_battery_current has changed for xmit_motor_current. 
    tech_argo_name["xmit_pump_battery_voltage"]="VOLTAGE_BatteryPumpStartProfile_volts" # with unit conversion. N.B units in .profile are wrong: to divide by 100.
    # Difficult to assess for this one also : the manual says xmit_pump_battery_voltage = "pump battery voltage in profile depth (during initialization of motor)".
    tech_argo_name["xmit_cpu_battery_voltage"]="VOLTAGE_BatteryCPUSurface_volts"
    # bet. case 2 and 4 the TECH had changed from Surface to Profile 
    # meaning the voltage measured at depth. Where is located the xmit_cpu_battery_voltage measurement: at surface or at depth ? The NEMO manual states: 
    # Byte 20: CPU battery voltage at surface and byte 21: Pump battery voltage in profile depth.
    tech_argo_name["xmit_serial_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_statusbyte"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_surface_pressure"]="PRES_SurfaceOffsetNotTruncated_dbar" # in this CASE 1, this is unsure: last pressure measured during ascent, 
    # reported @cycle c for cycle c or pressure measured before descent, reported @cycle c for cycle c-1 ?
    


In [24]:
if case == 2:
    # initiated from awi_0065 and awi_0064 : CASE 2
    print("Loading matching nemo to argo technical parameter names for case 2")
    tech_argo_name={}
    tech_argo_name["xmit_ascent_starttime"]="CLOCK_StartAscentToSurface_seconds"
    tech_argo_name["xmit_descent_starttime"]="CLOCK_StartDescentToProfile_seconds"
    tech_argo_name["xmit_surfacingtime"]="NORECORD" # In the manual: "Surfacing time at 100 dbar": this is not 
    # exactly "EndAscentToSurface". I have not found a tech term for this particular one.
    tech_argo_name["xmit_end_of_profile_time"]="CLOCK_EndAscentToSurface_seconds" # In the manual: "End of profile time 
    #(after last sample or abortion due to ice)"
    
    tech_argo_name["xmit_ice_detect_count"]="FLAG_IceDetected_COUNT"
    tech_argo_name["xmit_internal_vacuum"]="PRESSURE_InternalVacuumAtSurface_mbar" # With unit conversion
    tech_argo_name["xmit_park_piston_position"]="POSITION_PistonPark_COUNT"
    tech_argo_name["xmit_park_pressure_median"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, closest would be deprecated PRES_ParkMean_dbar
    tech_argo_name["xmit_piston_position_at_end"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, was previously recorded in POSITION_PistonSurface_COUNT ...
    tech_argo_name["xmit_piston_position_at_surf"]="POSITION_PistonSurface_COUNT"
    tech_argo_name["xmit_profile_length1"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length2"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length3"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length4"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length5"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_piston_position"]="POSITION_PistonProfile_COUNT"# maybe to change as TECH param defines as "piston position at maximum profile pressure"
    tech_argo_name["xmit_pump_battery_current"]="CURRENT_MotorStartProfile_amps" # with unit conversion. N.B units in .profile seem wrong (to multiply by 100 to get mamps or to divide by 10 to get amps), but not everywhere ...
    # Difficult to assess for this one: the manual says xmit_pump_battery_current = "hydraulic current during first ascent (during initialization of motor)".
    # the name has been changed between case 2 and 4: xmit_pump_battery_current has changed for xmit_motor_current. 
    tech_argo_name["xmit_pump_battery_voltage"]="VOLTAGE_BatteryPumpStartProfile_volts" # with unit conversion. N.B units in .profile are wrong: to divide by 100.
    # Difficult to assess for this one also : the manual says xmit_pump_battery_voltage = "pump battery voltage in profile depth (during initialization of motor)".
    tech_argo_name["xmit_cpu_battery_voltage"]="VOLTAGE_BatteryCPUSurface_volts"
    # bet. case 2 and 4 the TECH had changed from Surface to Profile 
    # meaning the voltage measured at depth. Where is located the xmit_cpu_battery_voltage measurement: at surface or at depth ? The NEMO manual states: 
    # Byte 20: CPU battery voltage at surface and byte 21: Pump battery voltage in profile depth.
    tech_argo_name["xmit_serial_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_statusbyte"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_surface_pressure"]="NORECORD" # in this CASE 2: last pressure measured during ascent, reported @cycle c for cycle c
    tech_argo_name["xmit_surface_pressure2"]="PRES_SurfaceOffsetNotTruncated_dbar" # in this CASE 2: pressure measured before descent, reported @cycle c for cycle c-1
    


In [25]:
if case == 3:
    # initiated from case 2
    print("Loading matching nemo to argo technical parameter names for case 3")
    tech_argo_name={}
    tech_argo_name["xmit_ascent_starttime"]="CLOCK_StartAscentToSurface_seconds"
    tech_argo_name["xmit_descent_starttime"]="CLOCK_StartDescentToProfile_seconds"
    tech_argo_name["xmit_surfacingtime"]="NORECORD" # In the manual: "Surfacing time at 100 dbar": this is not 
    # exactly "EndAscentToSurface". I have not found a tech term for this particular one.
    tech_argo_name["xmit_end_of_profile_time"]="CLOCK_EndAscentToSurface_seconds" # In the manual: "End of profile time 
    #(after last sample or abortion due to ice)"
    tech_argo_name["xmit_ice_detect_count"]="FLAG_IceDetected_COUNT"
    tech_argo_name["xmit_tube_pressure_surface"]="PRESSURE_InternalVacuumAtSurface_mbar" # With unit conversion
    tech_argo_name["xmit_park_piston_position"]="POSITION_PistonPark_COUNT"
    tech_argo_name["xmit_park_pressure_median"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, closest would be deprecated PRES_ParkMean_dbar
    tech_argo_name["xmit_piston_position_at_end"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, was previously recorded in POSITION_PistonSurface_COUNT ...
    tech_argo_name["xmit_piston_position_at_surf"]="POSITION_PistonSurface_COUNT"
    tech_argo_name["xmit_profile_length1"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length2"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length3"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length4"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length5"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_piston_position"]="POSITION_PistonProfile_COUNT"# maybe to change as TECH param defines as "piston position at maximum profile pressure"
    tech_argo_name["xmit_hydraulic_current_ascent"]="CURRENT_MotorStartProfile_amps" # with unit conversion. N.B units in .profile seem wrong (to multiply by 100 to get mamps or to divide by 10 to get amps), but not everywhere ...
    # Difficult to assess for this one: the manual says xmit_pump_battery_current = "hydraulic current during first ascent (during initialization of motor)".
    # the name has been changed between case 2 and 4: xmit_pump_battery_current has changed for xmit_motor_current. 
    tech_argo_name["xmit_pump_battery_voltage"]="VOLTAGE_BatteryPumpStartProfile_volts" # with unit conversion. N.B units in .profile are wrong: to divide by 100.
    # Difficult to assess for this one also : the manual says xmit_pump_battery_voltage = "pump battery voltage in profile depth (during initialization of motor)".
    tech_argo_name["xmit_cpu_battery_voltage"]="VOLTAGE_BatteryCPUSurface_volts"
    # bet. case 2 and 4 the TECH had changed from Surface to Profile 
    # meaning the voltage measured at depth. Where is located the xmit_cpu_battery_voltage measurement: at surface or at depth ? The NEMO manual states: 
    # Byte 20: CPU battery voltage at surface and byte 21: Pump battery voltage in profile depth.
    tech_argo_name["xmit_serial_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_statusbyte"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_surface_pressure1"]="NORECORD" # in this CASE 3: last pressure measured during ascent, reported @cycle c for cycle c
    tech_argo_name["xmit_surface_pressure2"]="PRES_SurfaceOffsetNotTruncated_dbar" # in this CASE 3: pressure measured before descent, reported @cycle c for cycle c-1



In [26]:
if case == 4:
    print("Loading matching nemo to argo technical parameter names for case 4")
    # initiated from nemo_0144 : CASE 4
    tech_argo_name={}
    tech_argo_name["xmit_descent_start_time"]="CLOCK_StartDescentToPark_seconds"
    tech_argo_name["xmit_parking_start_time"]="CLOCK_EndDescentToPark_seconds"
    tech_argo_name["xmit_upcast_start_time"]="CLOCK_StartDescentToProfile_seconds"
    tech_argo_name["xmit_ascent_start_time"]="CLOCK_StartAscentToSurface_seconds"
    tech_argo_name["xmit_surface_start_time"]="CLOCK_EndAscentToSurface_seconds"
    tech_argo_name["xmit_airpump_runtime"]="TIME_AirPumpOn_seconds" # are the units correct ?
    tech_argo_name["xmit_ascent_end_time"]="NORECORD"
    tech_argo_name["xmit_cpu_battery_voltage"]="VOLTAGE_BatteryCPUSurface_volts" # bet. case 2 and 4 the original TECH.nc had changed from Surface to Profile 
    # meaning the voltage measured at depth. Where is located the xmit_cpu_battery_voltage measurement: at surface or at depth ? The NEMO manual states: 
    # Byte 20: CPU battery voltage at surface and byte 21: Pump battery voltage in profile depth (see below).
    tech_argo_name["xmit_pump_battery_voltage"]="VOLTAGE_BatteryPumpStartProfile_volts" # "VOLTAGE_BatterySurfaceAirPumpOn_volts"
    # Difficult to assess for this one also : the manual says xmit_pump_battery_voltage = pump battery voltage in profile depth (during initialization of motor).
    tech_argo_name["xmit_motor_current"]="CURRENT_MotorStartProfile_amps"
    tech_argo_name["xmit_motor_current_mean"]="CURRENT_MotorMeanOfProfile_amps"
    tech_argo_name["xmit_motor_errors_count"]="NUMBER_MotorErrorDuringProfile_COUNT"
    tech_argo_name["xmit_depth_pressure"]="NORECORD"
    tech_argo_name["xmit_depth_pressure_max"]="NORECORD" # N.B. "PRES_DescentToProfileMaxPressure_dbar" is deprecated in 3.1
    tech_argo_name["xmit_expected_profile_time"]="NORECORD"
    tech_argo_name["xmit_external_battery_voltage"]="NORECORD"
    tech_argo_name["xmit_ice_det_temp_median"]="NORECORD"
    tech_argo_name["xmit_ice_detection_temp_median"]="NORECORD"
    tech_argo_name["xmit_icegps_time"]="NORECORD"
    tech_argo_name["xmit_iceirdm_time"]="NORECORD"
    tech_argo_name["xmit_internal_pressure_depth"]="PRESSURE_InternalVacuumProfileStart_mbar"
    tech_argo_name["xmit_internal_pressure_surface"]="PRESSURE_InternalVacuumAtSurface_mbar"
    tech_argo_name["xmit_internal_pressure_surface2"]="NORECORD"
    tech_argo_name["xmit_internal_temperature_surface"]="NORECORD"
    tech_argo_name["xmit_internal_temperature_surface2"]="NORECORD"
    tech_argo_name["xmit_o2_optode_errors"]="NUMBER_DOXYErrorsAscending_COUNT"
    tech_argo_name["xmit_older_profiles_not_sent"]="NUMBER_ProfileOldNotSent_COUNT"
    tech_argo_name["xmit_older_profiles_not_send"]="NUMBER_ProfileOldNotSent_COUNT"
    tech_argo_name["xmit_parking_pressure_median"]="NORECORD"
    tech_argo_name["xmit_piston_counts_calc"]="POSITION_PistonTarget_COUNT"
    tech_argo_name["xmit_piston_counts_depth"]="POSITION_PistonProfile_COUNT" #piston position at maximum profile pressure
    tech_argo_name["xmit_piston_counts_eop"]="NORECORD" # eop = end-of-profile (i.e. end of ascent). In practical: same value as xmit_piston_counts_surface
    tech_argo_name["xmit_piston_counts_parking"]="POSITION_PistonPark_COUNT"
    tech_argo_name["xmit_piston_counts_surface"]="POSITION_PistonSurface_COUNT"
    tech_argo_name["xmit_pressure_offset"]="NORECORD" # in this CASE 4: last pressure measured during ascent, reported @cycle c for cycle c
    tech_argo_name["xmit_surface_pressure"]="PRES_SurfaceOffsetNotTruncated_dbar" # in this CASE 4: pressure measured before descent, reported @cycle c for cycle c-1
    tech_argo_name["xmit_profile_length1"]="NORECORD"
    tech_argo_name["xmit_profile_length2"]="NORECORD"
    tech_argo_name["xmit_profile_length3"]="NORECORD"
    tech_argo_name["xmit_profile_length4"]="NORECORD"
    tech_argo_name["xmit_profile_length5"]="NORECORD"
    tech_argo_name["xmit_profile_lenght_1"]="NORECORD"
    tech_argo_name["xmit_profile_lenght_2"]="NORECORD"
    tech_argo_name["xmit_profile_lenght_3"]="NORECORD"
    tech_argo_name["xmit_profile_lenght_4"]="NORECORD"
    tech_argo_name["xmit_profile_lenght_5"]="NORECORD"
    tech_argo_name["xmit_profile_length_1"]="NORECORD"
    tech_argo_name["xmit_profile_length_2"]="NORECORD"
    tech_argo_name["xmit_profile_length_3"]="NORECORD"
    tech_argo_name["xmit_profile_length_4"]="NORECORD"
    tech_argo_name["xmit_profile_length_5"]="NORECORD"
    tech_argo_name["xmit_profile_number"]="NORECORD"
    tech_argo_name["xmit_profile_recovery"]="NORECORD"
    tech_argo_name["xmit_rafos_errors"]="NUMBER_RAFOSErrorsAscending_COUNT"
    tech_argo_name["xmit_sbe41_ctd_errors"]="NUMBER_CTDError_COUNT"
    tech_argo_name["xmit_sbe42_ctd_errors"]="NUMBER_CTDError_COUNT"
    tech_argo_name["xmit_serial_number"]="NORECORD"
    tech_argo_name["xmit_upcast_status"]="FLAG_IceDetected_LOGICAL" # It was previously stored as COUNT, but this is a logical in this case.
    tech_argo_name["xmit_surface_decetction_GPS"]="NORECORD"
    tech_argo_name["xmit_surface_decetction_Iridium"]="NORECORD"
    tech_argo_name["xmit_surface_detection_GPS"]="NORECORD"
    tech_argo_name["xmit_surface_detection_Iridium"]="NORECORD"

Loading matching nemo to argo technical parameter names for case 4


In [27]:
if case == 5:
    # initiated from case 2
    print("Loading matching nemo to argo technical parameter names for case 5")
    tech_argo_name={}
    tech_argo_name["xmit_ascent_starttime"]="CLOCK_StartAscentToSurface_seconds"
    tech_argo_name["xmit_descent_starttime"]="CLOCK_StartDescentToProfile_seconds"
    tech_argo_name["xmit_surfacingtime"]="NORECORD" # In the manual: "Surfacing time at 100 dbar": this is not 
    # exactly "EndAscentToSurface". I have not found a tech term for this particular one.
    tech_argo_name["xmit_end_of_profile_time"]="CLOCK_EndAscentToSurface_seconds" # In the manual: "End of profile time 
    #(after last sample or abortion due to ice)"
    tech_argo_name["xmit_ice_detect_count"]="FLAG_IceDetected_COUNT"
    tech_argo_name["xmit_internal_vacuum"]="PRESSURE_InternalVacuumAtSurface_mbar" # With unit conversion
    tech_argo_name["xmit_park_piston_position"]="POSITION_PistonPark_COUNT"
    tech_argo_name["xmit_park_pressure_median"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, closest would be deprecated PRES_ParkMean_dbar
    tech_argo_name["xmit_piston_position_at_end"]="NORECORD"#= NOT RECORDED in TECH file - no TECH param for that, was previously recorded in POSITION_PistonSurface_COUNT ...
    tech_argo_name["xmit_piston_position_at_surf"]="POSITION_PistonSurface_COUNT"
    tech_argo_name["xmit_profile_length1"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length2"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length3"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length4"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_length5"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_profile_piston_position"]="POSITION_PistonProfile_COUNT"# maybe to change as TECH param defines as "piston position at maximum profile pressure"
    tech_argo_name["xmit_pump_battery_current"]="CURRENT_MotorStartProfile_amps" # with unit conversion. N.B units in .profile seem wrong (to multiply by 100 to get mamps or to divide by 10 to get amps), but not everywhere ...
    # Difficult to assess for this one: the manual says xmit_pump_battery_current = "hydraulic current during first ascent (during initialization of motor)".
    # the name has been changed between case 2 and 4: xmit_pump_battery_current has changed for xmit_motor_current. 
    tech_argo_name["xmit_pump_battery_voltage"]="VOLTAGE_BatteryPumpStartProfile_volts" # with unit conversion. N.B units in .profile are wrong: to divide by 100.
    # Difficult to assess for this one also : the manual says xmit_pump_battery_voltage = "pump battery voltage in profile depth (during initialization of motor)".
    tech_argo_name["xmit_cpu_battery_voltage"]="VOLTAGE_BatteryCPUSurface_volts"
    # bet. case 2 and 4 the TECH had changed from Surface to Profile 
    # meaning the voltage measured at depth. Where is located the xmit_cpu_battery_voltage measurement: at surface or at depth ? The NEMO manual states: 
    # Byte 20: CPU battery voltage at surface and byte 21: Pump battery voltage in profile depth.
    tech_argo_name["xmit_serial_number"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_statusbyte"]="NORECORD"#= NOT RECORDED in TECH file
    tech_argo_name["xmit_offset_pressure"]="NORECORD"# in this CASE 3: xmit_offset_pressure=xmit_surface_pressure1
    tech_argo_name["xmit_surface_pressure1"]="NORECORD" # in this CASE 3: last pressure measured during ascent, reported @cycle c for cycle c
    tech_argo_name["xmit_surface_pressure2"]="PRES_SurfaceOffsetNotTruncated_dbar" # in this CASE 3: pressure measured before descent, reported @cycle c for cycle c-1


In [30]:
dac="coriolis"
#
wrk_dir="C:/Users/ddobler/Documents/08_DD_scripts/01_Scripts_Metadata_and_Data_Update/Techdata_WorkDir/"+lot+"/"
if not os.path.exists(wrk_dir):
    os.mkdir(wrk_dir)

#for wmo in list_wmo:
for iwmo in range(len(liste_wmos)):
#for iwmo in range(3):

    wmo=liste_wmos[iwmo]
    float_sn=liste_sn[iwmo]

    history_update_text='updated by D.Dobler (Euro-Argo ERIC)'

    # Select wmo to treat
    #wmo="6900203"
    print("\n\n---------------------------")
    print("Treating wmo ",wmo," float_sn: ",float_sn)

  
    # select modifications to perform

    i_convert_internal_vacuum_units_from_psi_to_mbar    = 0
    i_rebuild_internal_vacuum_from_dot_profile          = 0
    i_rebuild_pres_offset_from_dot_profile              = 0
    i_rebuild_BATT_pump_current_from_dot_profile        = 0
    i_rebuild_BATT_volt_from_dot_profile                = 0
    i_rebuild_FLAG_Ice_from_dot_profile                 = 0
    i_rebuild_for_all_param                             = 0 # do not use simultaneously with other i_rebuild selections.
    i_pump_battery_current_units_from_mampere_to_ampere = 0
    i_rename_ice_detection_param_name                   = 0
    i_switch_PRES_Now_PRESSURE_and_remove_PRES_Now      = 0
    i_bad_cycles_to_delete                              = 0
    bad_cycles_to_delete = [0]
    i_no_TPV                                            = 1
    i_cycles_to_keep_from_previous_tech_file            = 0 # for 7900224 and 7900219 for which I'm missing .profile in my archive.
    
    
    i_ftp_from_ifremer            = 1
    #where to save
    
    
    IN_FILE=wrk_dir + wmo + "_tech_ori.nc"
    OUT_FILE=wrk_dir + wmo + "_tech.nc"
    print(IN_FILE)

    
 
    # where to get 
    if i_ftp_from_ifremer:
        print("Getting file from data-argo.ifremer.fr")
        URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/"+wmo+"_tech.nc"
        print(URL)
        # commands
        response = requests.get(URL)
        open(IN_FILE, "wb").write(response.content)
    else:
        print("Getting file from local directory")
        #IN_FILE="C:/Users/ddobler/Documents/09_Scripts_WD/lot_014_deleted_level_in/" + wmo + "_meta.nc"
        #shutil.copyfile(IN_FILE,OUT_FILE)

    #print("Making changes for wmo ",wmo)

    ds_src=Dataset(IN_FILE,'r')
    ds_src.set_auto_mask(False)

    new_ds = Dataset(OUT_FILE, 'w',format='NETCDF3_CLASSIC')
    new_ds.set_auto_mask(False)

    for dimname, dim in ds_src.dimensions.items():
        if dimname == 'N_TECH_PARAM':
            new_ds.createDimension(dimname, None)
            N_TECH_PARAM=len(dim)
        else:
            new_ds.createDimension(dimname, len(dim))

    
    # update the global attribute history:
    date_courante=dt.datetime.now(dt.timezone.utc)
    date_courante_str=date_courante.strftime("%Y-%m-%dT%H:%M:%SZ")
    #lhistory=date_courante_str+ ' creation; '+date_courante_str+' last update (coriolis float real time data processing)'

    if i_no_TPV==0:
        TPN=np.array([None] * N_TECH_PARAM)
        TPV=np.array([None] * N_TECH_PARAM)

    
        CYCLE_NUMBER=ds_src.variables["CYCLE_NUMBER"][:]
        for i_tech_param in range(N_TECH_PARAM):
            TPN[i_tech_param]=ds_src.variables["TECHNICAL_PARAMETER_NAME"][i_tech_param,:].tobytes().decode().strip()
            TPV[i_tech_param]=ds_src.variables["TECHNICAL_PARAMETER_VALUE"][i_tech_param,:].tobytes().decode().strip()

        ## SECTION WHERE PARAMETER VALUES ARE REBUILD FROM .PROFILE INFORMATION
    
    
        # This is a patching part for missing .profile files in my archive directory.
        if i_cycles_to_keep_from_previous_tech_file:
            #CYCLE_NUMBER=CYCLE_NUMBER.astype('int')
            # 7900224 is missing cycles [16, 21, 26, 28,30]
            # 7900219 is missing cycles [2, 3,4,5,6,7,8,9,10,11,12,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]
            if (wmo == "7900224"):
                cycles_to_keep=np.array([16, 21, 26, 28,30])
            if (wmo == "7900219"):
                cycles_to_keep=np.array([2,3,4,5,6,7,8,9,10,11,12,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33])
            ic2k=np.nonzero(cycles_to_keep[:,None] == CYCLE_NUMBER)[1]
            CN_ic2k=CYCLE_NUMBER[ic2k]
            TPN_ic2k=TPN[ic2k]
            TPV_ic2k=TPV[ic2k]
    
        
        if i_rebuild_for_all_param:
            print("Rebuilding all tech_param from .profile")
            wrk_tech_files_dir="C:/Users/ddobler/Documents/NEMO/NEMO_profile_files/co010103v1/"
            file2load=wrk_tech_files_dir + "tech_dot_profile_" + float_sn + ".csv"
            
            print(file2load,os.path.exists(file2load))
            if os.path.exists(file2load):
                tech = pd.read_csv(file2load,sep=";",dtype=object) # type object is important otherwise values are converted to float
            else:
                print("ERROR: the file does not exist")
            lcycle=np.array(tech.cycle)
            lparam=np.array(tech.parameter)
            lvalue=np.array(tech.value,dtype=object)
            lunit=np.array(tech.unit)
        
            CYCLE_NUMBER,TPN,TPV=rebuild_complete_from_dot_profile(lcycle,lparam,lvalue,lunit)
            if i_cycles_to_keep_from_previous_tech_file:
                TPN=np.append(TPN,TPN_ic2k)
                TPV=np.append(TPV,TPV_ic2k)
                CYCLE_NUMBER=np.append(CYCLE_NUMBER,CN_ic2k)
                CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"TIME_ToAscend_SECONDS")
                TPN=change_TPN_name(TPN,"VOLTAGE_BatterySurfaceAirPumpOn_VOLTS","VOLTAGE_BatteryPumpStartProfile_volts")
            
    
        if i_rebuild_internal_vacuum_from_dot_profile:
            print("Rebuilding PRESSURE_InternalVacuumAtSurface_mbar from .profile")
            # The first idea was to keep one out of two but it seems that it was not always ordered the same way.
            # Thus I decided to remove all entries and reconstruct one from the .profile file.
            CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"PRESSURE_InternalVacuumAtSurface_mbar")
            TPN=np.append(TPN,Internal_vacuum_names)
            TPV=np.append(TPV,Internal_vacuum_values)
            CYCLE_NUMBER=np.append(CYCLE_NUMBER,Internal_vacuum_cycles)
    
        if i_rebuild_pres_offset_from_dot_profile:
            print("Rebuilding PRES_SurfaceOffsetNotTruncated_dBAR from .profile")
            CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"PRES_SurfaceOffsetNotTruncated_dBAR")
            TPN=np.append(TPN,PRES_offset_names)
            TPV=np.append(TPV,PRES_offset_values)
            CYCLE_NUMBER=np.append(CYCLE_NUMBER,PRES_offset_cycles)
            
        if i_rebuild_BATT_pump_current_from_dot_profile:
            print("Rebuilding Battery_pump_current from .profile")
            CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"CURRENT_BatterySurfaceAirPumpOn_mAMPS")
            TPN=np.append(TPN,BATT_pump_current_names)
            TPV=np.append(TPV,BATT_pump_current_values)
            CYCLE_NUMBER=np.append(CYCLE_NUMBER,BATT_pump_current_cycles)
    
        if i_rebuild_BATT_volt_from_dot_profile:
            print("Rebuilding VOLTAGE_BatteryCPUSurface_volts from .profile")
            CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"VOLTAGE_BatteryCPUSurface_VOLTS")
            TPN=np.append(TPN,BATT_volt_names)
            TPV=np.append(TPV,BATT_volt_values)
            CYCLE_NUMBER=np.append(CYCLE_NUMBER,BATT_volt_cycles)
    
        if i_rebuild_FLAG_Ice_from_dot_profile:
            print("Rebuilding FLAG_IceDetected_COUNT from .profile")
            CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"FLAG_IceDetected_COUNT")
            TPN=np.append(TPN,FLAG_Ice_names)
            TPV=np.append(TPV,FLAG_Ice_values)
            CYCLE_NUMBER=np.append(CYCLE_NUMBER,FLAG_Ice_cycles)
    
    
        if i_bad_cycles_to_delete:
            print("Deleting bad cycles")
            CYCLE_NUMBER,TPN,TPV=delete_cycles(CYCLE_NUMBER,TPN,TPV,bad_cycles_to_delete)
            #print(CYCLE_NUMBER)
    
        
            
            
            #for i_entry in range(60):
            #    print(CYCLE_NUMBER[i_entry],TPN[i_entry],TPV[i_entry])
        if i_convert_internal_vacuum_units_from_psi_to_mbar:
            print("converting internal vacuum units")
            i_TPN_Vacuum  = [index for (index, item) in enumerate(list(TPN)) if item == "PRESSURE_InternalVacuumAtSurface_mbar"]
            TPVVacuum=TPV[i_TPN_Vacuum].astype('float')
            TPVVacuum_mbar=TPVVacuum*68.9475729318
            TPV[i_TPN_Vacuum]=TPVVacuum_mbar[:]
    
        if i_pump_battery_current_units_from_mampere_to_ampere:
            print("converting BatterySurfaceAirPumpOn units from milliampere to ampere")
            i_TPN  = [index for (index, item) in enumerate(list(TPN)) if item == "CURRENT_BatterySurfaceAirPumpOn_mAMPS"]
            TPV_val_mamp=TPV[i_TPN].astype('float')
            TPV_val_amp=TPV_val_mamp/1000
            TPV[i_TPN]=TPV_val_amp[:]
            TPN[i_TPN]="CURRENT_BatterySurfaceAirPumpOn_amps"
    
            #for i_entry in i_TPN_Vacuum:
            #    print(CYCLE_NUMBER[i_entry],TPN[i_entry],TPV[i_entry])
    
        # A few parameter_units modification to be compliant with 3.1:
        TPN=change_TPN_name(TPN,"CLOCK_EndAscentToSurface_SECONDS","CLOCK_EndAscentToSurface_seconds")
        TPN=change_TPN_name(TPN,"CLOCK_FloatTime_SECONDS","CLOCK_FloatTime_seconds")
        TPN=change_TPN_name(TPN,"CLOCK_StartAscentToSurface_SECONDS","CLOCK_StartAscentToSurface_seconds")
        TPN=change_TPN_name(TPN,"CLOCK_StartDescentToProfile_SECONDS","CLOCK_StartDescentToProfile_seconds")
        TPN=change_TPN_name(TPN,"TIME_ToAscend_SECONDS","TIME_ToAscend_seconds")
        TPN=change_TPN_name(TPN,"VOLTAGE_BatteryCPUSurface_VOLTS","VOLTAGE_BatteryCPUSurface_volts")
        TPN=change_TPN_name(TPN,"VOLTAGE_BatterySurfaceAirPumpOn_VOLTS","VOLTAGE_BatterySurfaceAirPumpOn_volts")
        TPN=change_TPN_name(TPN,"PRES_SurfaceOffsetNotTruncated_dBAR","PRES_SurfaceOffsetNotTruncated_dbar")
    
        #special case for DDMMYYYY -> YYYYMMDD
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_EndAscentToSurface_DDMMYYYYHHMMSS", \
                                             "CLOCK_EndAscentToSurface_YYYYMMDDHHMMSS", \
                                             "DD/MM/YYYY HH:MM:SS","YYYYMMDDHHMMSS")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_StartAscentToSurface_DDMMYYYYHHMMSS", \
                                             "CLOCK_StartAscentToSurface_YYYYMMDDHHMMSS", \
                                             "DD/MM/YYYY HH:MM:SS","YYYYMMDDHHMMSS")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_StartDescentProfile_DDMMYYYYHHMMSS", \
                                             "CLOCK_StartDescentProfile_YYYYMMDDHHMMSS", \
                                             "DD/MM/YYYY HH:MM:SS","YYYYMMDDHHMMSS")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_TransmissionStart_DDMMYYYYHHMMSS", \
                                             "CLOCK_TransmissionStart_YYYYMMDDHHMMSS", \
                                             "DD/MM/YYYY HH:MM:SS","YYYYMMDDHHMMSS")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_FloatTime_HHMMSS", \
                                             "CLOCK_FloatTime_HHMM", \
                                             "HH:MM","HHMM")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_FloatTime_HHMMSS", \
                                             "CLOCK_FloatTime_YYYYMMDDHHMMSS", \
                                             "YYYY/MM/DD HH:MM","YYYYMMDDHHMMSS")
        TPN,TPV=change_TPN_date_unit(TPN,TPV,"CLOCK_Satellite_HHMMSS", \
                                             "CLOCK_Satellite_HHMM", \
                                             "HH:MM","HHMM")
        
        # Deprecated parameters:
        CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"CONFIG_InstReference_STRING")
        CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"CONFIG_AscendingProfilingTime_HHMM")
        CYCLE_NUMBER,TPN,TPV=delete_TP(CYCLE_NUMBER,TPN,TPV,"CONFIG_ParkPressure_dBAR") # This will be put in trajectory.
    
        
        
        # Once all modifications are performed: sorting array (by cycle_number and by TPN)
        N_TECH_PARAM_new=np.size(CYCLE_NUMBER)
        sorting_argument_str_format=[None]*N_TECH_PARAM_new
        #print(CYCLE_NUMBER.shape,TPN.shape,N_TECH_PARAM_new)
        for i in range(N_TECH_PARAM_new):
            sorting_argument_str_format[i]="{0:04d}".format(CYCLE_NUMBER[i]) + TPN[i]
        i_sort=np.argsort(sorting_argument_str_format)
        CYCLE_NUMBER=CYCLE_NUMBER[i_sort]
        TPN=TPN[i_sort]
        TPV=TPV[i_sort]
    
    

    # creating the new tech file:
    print("Creating the new tech.nc file with :")
    print(str(N_TECH_PARAM_new) + " lines instead of " + str(N_TECH_PARAM))
    for varname, var in ds_src.variables.items():
        print("\n Treating ", varname)

        new_var = new_ds.createVariable(varname, var.dtype, var.dimensions, fill_value=ds_src.variables[varname]._FillValue)

        for local_attrs in ds_src.variables[varname].ncattrs():
            if (local_attrs != '_FillValue'):
                    new_var.setncattr(local_attrs,ds_src.variables[varname].getncattr(local_attrs))

        # modifications toward 3.1 format:
        if varname == "DATA_TYPE":
            new_var.setncattr('conventions',"Argo reference table 1")
        if varname == "CYCLE_NUMBER":
            new_var.setncattr('conventions',"0...N, 0 : launch cycle (if exists), 1 : first complete cycle")
        
        
        if varname not in ['CYCLE_NUMBER','TECHNICAL_PARAMETER_NAME','TECHNICAL_PARAMETER_VALUE']:
            print("recording ",varname)
            if varname == "FORMAT_VERSION":
                change_val_1D_char(new_ds,"FORMAT_VERSION",'3.1')
            else:
                new_var[:]=ds_src.variables[varname][:]

        elif i_no_TPV == 0:
            print("recording ",varname)
            if varname == "CYCLE_NUMBER":
                new_var[:]=CYCLE_NUMBER[:]
            if varname == "TECHNICAL_PARAMETER_NAME":
                for jj in range(N_TECH_PARAM_new):
                    new_val=TPN[jj]
                    for i in range(len(new_val)):
                        new_var[jj,i]=new_val[i]
            if varname == "TECHNICAL_PARAMETER_VALUE":
                for jj in range(N_TECH_PARAM_new):
                    new_val=str(TPV[jj])
                    for i in range(len(new_val)):
                        new_var[jj,i]=new_val[i]
                        
    
    print("Completing global attribute section")
    new_ds.title = "Argo float technical data file"
    new_ds.institution = "CORIOLIS"
    new_ds.source = "Argo float"
    new_ds.history = ""
    new_ds.references = "http://www.argodatamgt.org/Documentation"
    try:
        new_ds.comment = ds_src.comment
    except:
        print("No comment attribute among global attributes of initial tech file")
    new_ds.user_manual_version = "3.4"
    new_ds.Conventions = "Argo-3.1 CF-1.6"
    try:
        new_ds.decoder_version = ds_src.decoder_version
    except:
        print("No decoder_version attribute among global attributes of initial tech file")
    new_ds.id = "https://doi.org/10.17882/42182"


    #print("check")
    #print(ds.history)
    
    print("\n Updating history global attribute")

    tmp=str(ds_src.variables["DATE_CREATION"][:].tobytes(),'utf-8')
    record_date_creation=tmp[:4]+"-"+tmp[4:6]+"-"+tmp[6:8]+"T"+tmp[8:10]+":"+tmp[10:12]+":"+tmp[12:14]+"Z"
    #print(record_date_creation)


    # update the global attribute history:
    date_courante=dt.datetime.now(dt.timezone.utc)
    date_courante_str=date_courante.strftime("%Y-%m-%dT%H:%M:%SZ")
    lhistory=record_date_creation + ' creation; ' + date_courante_str + ' ' + history_update_text 
    #lhistory=record_date_creation+ ' creation; '+date_courante_str+' updated'
    try:
        print("Old history global attr: ",ds_src.history)
    except:
        print("Old history global attr did not exist")
    new_ds.history = lhistory
    print("New history global attr: ",new_ds.history)

    print("Updating DATE_UPDATE")
    varname="DATE_UPDATE"
    new_val=date_courante.strftime("%Y%m%d%H%M%S")
    change_val_1D_char(new_ds,varname,new_val)
    

    new_ds.close()
    ds_src.close()



---------------------------
Treating wmo  1900380  float_sn:  1900380
C:/Users/ddobler/Documents/08_DD_scripts/01_Scripts_Metadata_and_Data_Update/Techdata_WorkDir/lot_015/1900380_tech_ori.nc
Getting file from data-argo.ifremer.fr
https://data-argo.ifremer.fr/dac/coriolis/1900380/1900380_tech.nc
Creating the new tech.nc file with :
38 lines instead of 41

 Treating  PLATFORM_NUMBER
recording  PLATFORM_NUMBER

 Treating  DATA_TYPE
recording  DATA_TYPE

 Treating  FORMAT_VERSION
recording  FORMAT_VERSION
Modifying  FORMAT_VERSION  from    to  3.1

 Treating  HANDBOOK_VERSION
recording  HANDBOOK_VERSION

 Treating  DATA_CENTRE
recording  DATA_CENTRE

 Treating  DATE_CREATION
recording  DATE_CREATION

 Treating  DATE_UPDATE
recording  DATE_UPDATE

 Treating  TECHNICAL_PARAMETER_NAME

 Treating  TECHNICAL_PARAMETER_VALUE

 Treating  CYCLE_NUMBER
Completing global attribute section
No decoder_version attribute among global attributes of initial tech file

 Updating history global attribute

In [ ]:
print(TPN)